In [ ]:
from Measurement import KineticEnergy, PotentialEnergy, ParticleNumberFluctuations, Energy, ParticleNumberFluctuationsFromCorrelationsPBC
from Result import Result, Parameters
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np 
import zipfile_deflate64 as z64

from tqdm.auto import tqdm
from scipy.optimize import curve_fit

Load the raw data and compute averages and errors.

In [ ]:
def load_qmc(L:int,seed_array=np.arange(1,1001,1),n_bins_threshold=350, beta_array = None ):
    path = lambda beta, seed: Path(f"./raw/sigma2_1D_L{L}_N{L}_M{L}_l{L//2}_U3.2255_t1.0000_beta{beta:4.4f}_binsize10000_seed{seed}_square_trialstate_gutzwiller.zip") 
    
    if beta_array is None:
        beta_array = np.array([L/2])
    
    for beta in beta_array:
        fn = path(beta,5000)
        print(fn.stem)
        params = Parameters.from_filename(fn.stem)
        break
        
    result = Result(params, path=path) 
    result.add_measurement(measurement_factory=ParticleNumberFluctuationsFromCorrelationsPBC)
        
    for beta in beta_array:
        fn = path(beta, 5000)
        zip = z64.ZipFile(fn) 
        for seed in tqdm(seed_array):
            try:
                try:
                    result.add_seed(seed=seed, beta=beta, n_bins_threshold=n_bins_threshold, zip=zip) 
                except ValueError as e:
                    print(f"Skipping seed {seed} due to: ", e)
            except FileNotFoundError:
                pass
                print(f"Skipping seed {seed} as file was not found.") 
    result.evaluate()
    return result

In [ ]:
L_array = [32,48,96]
beta_arrays = [[6,8,10,12,14,18,20],[10,15,20,24,30],[32,36,40,48]]

pigsfli = {}
for L, beta_array in zip(L_array,beta_arrays):
    pigsfli[L] = load_qmc(L, seed_array=np.arange(1,5001),beta_array=beta_array)

Perform the beta scaling

In [ ]:
def beta_scale_F(F_dict,L):
    beta_array = np.array(list(F_dict.keys()))
    Fs = np.array([F_dict[beta][0] for beta in beta_array])
    dFs = np.array([F_dict[beta][1] for beta in beta_array])
    
     
    beta_law = lambda beta, F_inf, c1, c2: F_inf + c1 * np.exp(-c2*beta) 

    F_inf = []
    dF_inf = []
    c1 = []
    c2 = []

    for i in range(L//2): 
        F = np.array([F[i] for F in Fs])
        dF = np.array([dF[i] for dF in dFs])
        betas = np.array(beta_array) 
        
        popt,pcov = curve_fit(beta_law, betas, F, absolute_sigma=True, sigma=dF, p0=[F[-1], 5, 1e-5],maxfev =100000,bounds=([0,-np.inf,0],[5,np.inf,10]))
         
        F_inf.append(popt[0])
        dF_inf.append(np.sqrt(np.diag(pcov))[0])
        c1.append(popt[1])
        c2.append(popt[2])    
         
    F_inf = np.array(F_inf)
    dF_inf = np.array(dF_inf)
    
    return np.arange(1,L//2+1), F_inf, dF_inf

In [ ]:
ls = {}
F = {}
dF = {}
for L in L_array: 
    ls[L], F[L], dF[L] = beta_scale_F(pigsfli[L].get_result("F_corr_pbc"),L=L) 

Write aggregated data to file

In [ ]:
for L in L_array:  
    np.save(f"../sigma2_2_aggregated_L{L}_U3.2255.npy",np.array([ls[L],F[L],dF[L]]).T)